# 03 - SLAM Evaluation

Evaluate the SLAM pipeline: odometry accuracy, loop closure detection, trajectory comparison.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.utils.io_utils import load_config, load_trajectory
from src.evaluation.metrics import compute_ate, compute_rpe
from src.evaluation.visualization import (
    plot_trajectory_2d,
    plot_trajectory_3d,
    plot_error_over_time,
    plot_recall_at_k,
)

## 1. Load Results

In [ ]:
# Load trajectories from evaluation results
# results_dir = Path('../results/2012-01-08')
# estimated = load_trajectory(results_dir / 'estimated.tum', format='tum')
# ground_truth = load_trajectory(results_dir / 'ground_truth.tum', format='tum')
# print(f'Estimated: {estimated.shape}')
# print(f'Ground truth: {ground_truth.shape}')

# Demo with synthetic data
N = 200
t = np.linspace(0, 4 * np.pi, N)
gt_poses = np.zeros((N, 4, 4))
est_poses = np.zeros((N, 4, 4))

for i in range(N):
    gt_poses[i] = np.eye(4)
    gt_poses[i, 0, 3] = 50 * np.cos(t[i])
    gt_poses[i, 1, 3] = 50 * np.sin(t[i])
    gt_poses[i, 2, 3] = 0.1 * t[i]
    
    est_poses[i] = gt_poses[i].copy()
    # Add drift
    est_poses[i, 0, 3] += 0.02 * i + np.random.randn() * 0.5
    est_poses[i, 1, 3] += 0.01 * i + np.random.randn() * 0.5

print(f'Synthetic trajectory: {N} poses')

## 2. Compute Metrics

In [ ]:
ate = compute_ate(est_poses, gt_poses)
rpe = compute_rpe(est_poses, gt_poses, delta=1)

print('Absolute Trajectory Error (ATE):')
for key, val in ate.items():
    print(f'  {key}: {val:.4f} m')

print('\nRelative Pose Error (RPE):')
for key, val in rpe.items():
    unit = 'deg' if 'rot' in key else 'm'
    print(f'  {key}: {val:.4f} {unit}')

## 3. Trajectory Visualization

In [ ]:
fig = plot_trajectory_2d(
    {'Ground Truth': gt_poses, 'Estimated (with drift)': est_poses},
    title='Trajectory Comparison (Synthetic Demo)',
)
plt.show()

In [ ]:
fig = plot_trajectory_3d(
    {'Ground Truth': gt_poses, 'Estimated': est_poses},
    title='3D Trajectory Comparison',
)
plt.show()

## 4. Per-Frame Error Analysis

In [ ]:
# Compute per-frame translation errors
from src.evaluation.metrics import umeyama_alignment

pos_est = est_poses[:, :3, 3]
pos_gt = gt_poses[:, :3, 3]

R, t_align, s = umeyama_alignment(pos_est, pos_gt)
pos_aligned = s * (R @ pos_est.T).T + t_align
per_frame_errors = np.linalg.norm(pos_aligned - pos_gt, axis=1)

fig = plot_error_over_time(
    per_frame_errors,
    title='Per-Frame ATE After Alignment',
    ylabel='Error (m)',
)
plt.show()